In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox
import re

In [5]:

def clean_roll_number(roll):
    return re.sub(r'\s*[Gg]\d+', '', str(roll)).strip()

def add_event():
    event_name = event_entry.get()
    event_hours = hours_entry.get()
    
    if not event_name or not event_hours:
        messagebox.showerror("Input Error", "Please enter both event name and hours.")
        return
    
    try:
        event_hours = float(event_hours)
    except ValueError:
        messagebox.showerror("Input Error", "Hours must be a number.")
        return
    
    main_file = "SEM2-HOURS.xlsx"  # Fixed main file
    df = pd.read_excel(main_file, sheet_name=0)
    
    if event_name in df.columns:
        messagebox.showerror("Duplicate Event", "This event already exists.")
        return
    
    df[event_name] = 0  # Add new event column with default 0 hours
    df.loc[0, event_name] = event_hours  # Set event hours in the first row
    
    df.to_excel(main_file, index=False)
    
    messagebox.showinfo("Success", f"Event '{event_name}' added successfully!")

def update_attendance():
    attendance_file = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv"), ("Excel Files", "*.xlsx")])
    if not attendance_file:
        return
    
    main_file = "SEM2-HOURS.xlsx"  # Fixed main file
    df = pd.read_excel(main_file, sheet_name=0)
    last_event = list(df.columns)[-1]  # Last event column
    event_hours = df.loc[0, last_event]  # Hours assigned to the event
    
    if attendance_file.endswith(".csv"):
        attendance_df = pd.read_csv(attendance_file)
    else:
        attendance_df = pd.read_excel(attendance_file, sheet_name=0)
    
    # Ensure roll number column exists
    roll_number_columns = ["ROLL NO", "Roll Number", "Student ID"]
    found_column = None
    for col in roll_number_columns:
        if col in attendance_df.columns:
            found_column = col
            break
    
    if not found_column:
        messagebox.showerror("Error", "No valid roll number column found in the attendance file.")
        return
    
    # Clean roll numbers in attendance file
    attendance_df[found_column] = attendance_df[found_column].astype(str).apply(clean_roll_number)
    present_ids = set(attendance_df[found_column])
    
    # Clean roll numbers in main file
    df['ROLL NO'] = df['ROLL NO'].astype(str).apply(clean_roll_number)
    
    df.loc[df['ROLL NO'].isin(present_ids), last_event] += event_hours
    
    df.to_excel(main_file, index=False)
    
    messagebox.showinfo("Success", "Attendance updated successfully!")

# GUI Setup
root = tk.Tk()
root.title("Generalized Attendance Manager")

tk.Label(root, text="Event Name:").grid(row=0, column=0)
event_entry = tk.Entry(root)
event_entry.grid(row=0, column=1)

tk.Label(root, text="Hours:").grid(row=1, column=0)
hours_entry = tk.Entry(root)
hours_entry.grid(row=1, column=1)

tk.Button(root, text="Add Event", command=add_event).grid(row=2, column=0, columnspan=2)
tk.Button(root, text="Update Attendance", command=update_attendance).grid(row=3, column=0, columnspan=2)

root.mainloop()


In [ ]:
2401EC03
2401EC09
2401mc43
2401CB13
2401CS30
2401ES04
2401CS48
2401AI21